In [0]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Note: using Google CoLab


In [0]:
!wget https://www.dropbox.com/s/64k2kcl8nhx22z6/clips-data-2020.zip?dl=0
!mv clips-data-2020.zip?dl=0 clips-data-2020.zip
!unzip -o clips-data-2020.zip > /dev/null
print("complete")

--2020-04-12 00:34:10--  https://www.dropbox.com/s/64k2kcl8nhx22z6/clips-data-2020.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/64k2kcl8nhx22z6/clips-data-2020.zip [following]
--2020-04-12 00:34:10--  https://www.dropbox.com/s/raw/64k2kcl8nhx22z6/clips-data-2020.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc244828220ecf4e2095a48ec70e.dl.dropboxusercontent.com/cd/0/inline/A1qwnpyo0Y0_RK5tqBPz46orRJ2MTRDFA0Qcoq1jKBaQEpKhjU1_8i-u-hP8E7n0toV-lDJHH7OeDVBTOVcaMZ4i0VFsaIayKWskFld4v65YBqOo_XnnrQN0KMdCHw0stMI/file# [following]
--2020-04-12 00:34:11--  https://uc244828220ecf4e2095a48ec70e.dl.dropboxusercontent.com/cd/0/inline/A1qwnpyo0Y0_RK5tqBPz46orRJ2MTRDFA0Qcoq1jKBaQEpKhjU1_8i-u-hP8E7n0toV-lDJHH7OeDVBTOVcaM

Next, we build a dataframe that contains the filenames and the clip_count.  You will need to modify the paths below to match where you put train.csv and master.csv.

In [0]:
import pandas as pd

df0 = pd.read_csv(
    '/content/drive/My Drive/KaggleProject/train.csv', 
    na_values=['NA', '?'])
df1 = pd.read_csv(
    '/content/drive/My Drive/KaggleProject/master.csv', 
    na_values=['NA', '?'])
df = pd.concat([df0,df1])

df['filename']="clips-"+df["id"].astype(str)+".png"

Separate into a training and validation (for early stopping)

In [0]:
TRAIN_PCT = 0.75
TRAIN_CUT = int(len(df) * TRAIN_PCT)

df_train = df[0:TRAIN_CUT]
df_validate = df[TRAIN_CUT:]

print(f"Training size: {len(df_train)}")
print(f"Validate size: {len(df_validate)}")

Training size: 33750
Validate size: 11250


Create DataGenerators for training and validation.  This is what links the .csv with the images.  This part might take a few minutes to run.

In [0]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

IMAGES_DIR = "/content/clips"

training_datagen = ImageDataGenerator(
  samplewise_center = True,
  rescale = 1./255,
  horizontal_flip=True,
  vertical_flip=True,
  fill_mode = 'nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(256, 256),
        batch_size=16,  
        class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255, samplewise_center = True)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate,
        directory=IMAGES_DIR,
        x_col="filename",
        y_col="clip_count",
        target_size=(256, 256),
        class_mode='other')

Found 33750 validated image filenames.
Found 11250 validated image filenames.


In [0]:
from tensorflow.keras.callbacks import EarlyStopping

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(256, 256, 3)), 
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])


model.summary()
epoch_steps = 2000 
validation_steps = len(df_validate)
model.compile(loss = 'mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=6, verbose=1, mode='auto',
        restore_best_weights=True) 
history = model.fit(train_generator,  
  steps_per_epoch = epoch_steps, 
  validation_steps = validation_steps, verbose = 1, 
  callbacks=[monitor], validation_data=val_generator, epochs=40)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 246016)            0         
_________________________________________________________________
dense (Dense)                (None, 512)               125960704 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5

# Generate Submission File

In [0]:
df_test = pd.read_csv(
    '/content/drive/My Drive/KaggleProject/test.csv', 
    na_values=['NA', '?'])

df_test['filename']="clips-"+df_test["id"].astype(str)+".png"

test_datagen = ImageDataGenerator(rescale = 1./255, samplewise_center = True)

test_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=IMAGES_DIR,
        x_col="filename",
        batch_size=1,
        shuffle=False,
        target_size=(256, 256),
        class_mode=None)

Found 5000 validated image filenames.


In [0]:
test_generator.reset()
pred = model.predict(test_generator,steps=len(df_test))
pred

array([[ 2.3331058],
       [48.81033  ],
       [63.52879  ],
       ...,
       [60.25589  ],
       [ 5.5525494],
       [32.006855 ]], dtype=float32)

In [0]:
df_submit = pd.DataFrame({'id':df_test['id'],'clip_count':pred.flatten()})
df_submit

,id,clip_count
0,25001,2.333106
1,25002,48.810329
2,25003,63.528790
3,25004,2.789717
4,25005,44.849838
...,...,...
4995,29996,30.295801
4996,29997,56.102882
4997,29998,60.255890
4998,29999,5.552549


In [0]:
df_submit.to_csv("/content/drive/My Drive/KaggleProject/submit7.csv",index=False)